In [1]:
!pip install bs4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from bs4 import BeautifulSoup as bs
import requests
from re import search
import re
import pprint

In [61]:
url = 'https://barnaul.hh.ru/search/vacancy'
#https://barnaul.hh.ru/search/vacancy?text=data+scientist&area=2&items_on_page=20&page=0
#https://barnaul.hh.ru/search/vacancy?&text=data+scientist&area=2&items_on_page=20&page=0
params = {
    'area': 2,
  #  'search_field': 'description',
   # 'search_field': 'company_name',
    'text': 'Data scientist',
    'items_on_page': 20,
    'page': 0,

}
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'}
vacancies_list = []
i = 0
while True:
  response = requests.get(url=url, params=params, headers=headers)
  soup = bs(response.text, 'html.parser')
  vacancies = soup.find_all('div', {'class': 'serp-item'})
  for vacancy in vacancies:
    vacansy_name = vacancy.find('a', {'class': 'serp-item__title'}).getText()
    vacansy_link = vacancy.find('a', {'class': 'serp-item__title'})['href']
    vacansy_salary = vacancy.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'})
    if vacansy_salary is not None:
      vacansy_salary = vacansy_salary.getText()
      if search('до', vacansy_salary):
        salary_list = re.findall(r'\d+', vacansy_salary) 
        max_salary = salary_list[0] + salary_list[1]
        min_salary = None
      elif search('от', vacansy_salary):
        salary_list = re.findall(r'\d+', vacansy_salary)
        min_salary = salary_list[0] + salary_list[1]
        max_salary = None
      elif search(' – ', vacansy_salary):
        salary_list = re.findall(r'\d+', vacansy_salary)
        if len(salary_list) < 3:
          min_salary = salary_list[0] 
          max_salary =  salary_list[1] 
        else:
          min_salary = salary_list[0] + salary_list[1]
          max_salary = salary_list[2] + salary_list[3]
    else:
      max_salary = None
      min_salary = None
    i = i + 1
    vacancy_dict = {
      'N': i,
      'name': vacansy_name,
      'link': vacansy_link,
      'min_salary': min_salary,
      'max_salary': max_salary,
      }
    vacancies_list.append(vacancy_dict)
  if len(vacancies) > 0:
    params['page'] = params['page'] + 1  
  else:
    break


In [62]:
vacancies_list

[{'N': 1,
  'name': 'Data Scientist',
  'link': 'https://barnaul.hh.ru/vacancy/70360322?from=vacancy_search_list&hhtmFrom=vacancy_search_list&query=Data%20scientist',
  'min_salary': None,
  'max_salary': '105000'},
 {'N': 2,
  'name': 'Data Scientist',
  'link': 'https://barnaul.hh.ru/vacancy/70483879?from=vacancy_search_list&hhtmFrom=vacancy_search_list&query=Data%20scientist',
  'min_salary': None,
  'max_salary': None},
 {'N': 3,
  'name': 'Data Scientist',
  'link': 'https://barnaul.hh.ru/vacancy/69651789?from=vacancy_search_list&hhtmFrom=vacancy_search_list&query=Data%20scientist',
  'min_salary': None,
  'max_salary': None},
 {'N': 4,
  'name': 'Data scientist/Antifraud analyst',
  'link': 'https://barnaul.hh.ru/vacancy/68898421?from=vacancy_search_list&hhtmFrom=vacancy_search_list&query=Data%20scientist',
  'min_salary': None,
  'max_salary': None},
 {'N': 5,
  'name': 'Риск-менеджер (Data Scientist)',
  'link': 'https://barnaul.hh.ru/vacancy/70514581?from=vacancy_search_list&h

In [63]:
type(vacancies_list)

list

In [14]:
response.status_code

200

In [64]:
import pandas as pd


In [66]:
print(pd.DataFrame(vacancies_list))

     N                                               name  \
0    1                                     Data Scientist   
1    2                                     Data Scientist   
2    3                                     Data Scientist   
3    4                   Data scientist/Antifraud analyst   
4    5                     Риск-менеджер (Data Scientist)   
5    6                              Middle Data Scientist   
6    7                                 Data Scientist CPA   
7    8                              Senior Data Scientist   
8    9                                     Data Scientist   
9   10                                     Data Scientist   
10  11                   Data Scientist (Computer Vision)   
11  12                                     Data Scientist   
12  13                  Data scientist (проект "Моменты")   
13  14                        Senior Data Scientist (WAF)   
14  15  Data scientist / Machine learning engineer (ко...   
15  16                 D